# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [4]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Load the dataset
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")

# Feature Selection: Drop unnecessary columns
columns_to_drop = ["PassengerId", "Name"]
spaceship = spaceship.drop(columns=[col for col in columns_to_drop if col in spaceship.columns], axis=1)

# Convert categorical features into numerical using one-hot encoding
spaceship = pd.get_dummies(spaceship, columns=["HomePlanet", "CryoSleep", "Cabin", "Destination", "VIP"], drop_first=True)

# Handle missing values using median imputation for numerical columns
num_cols = ["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
imputer = SimpleImputer(strategy="median")
spaceship[num_cols] = imputer.fit_transform(spaceship[num_cols])

# Feature Scaling: Apply StandardScaler to numerical features
scaler = StandardScaler()
spaceship[num_cols] = scaler.fit_transform(spaceship[num_cols])

# Display processed data
print(spaceship.head())  # Show the first 5 rows of the processed dataset

        Age  RoomService  FoodCourt  ShoppingMall       Spa    VRDeck  \
0  0.711945    -0.333105  -0.281027     -0.283579 -0.270626 -0.263003   
1 -0.334037    -0.168073  -0.275387     -0.241771  0.217158 -0.224205   
2  2.036857    -0.268001   1.959998     -0.283579  5.695623 -0.219796   
3  0.293552    -0.333105   0.523010      0.336851  2.687176 -0.092818   
4 -0.891895     0.125652  -0.237159     -0.031059  0.231374 -0.261240   

   Transported  HomePlanet_Europa  HomePlanet_Mars  CryoSleep_True  ...  \
0        False               True            False           False  ...   
1         True              False            False           False  ...   
2        False               True            False           False  ...   
3        False               True            False           False  ...   
4         True              False            False           False  ...   

   Cabin_G/999/P  Cabin_G/999/S  Cabin_T/0/P  Cabin_T/1/P  Cabin_T/2/P  \
0          False          False     

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [5]:
# Import necessary libraries
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Assuming 'Transported' is the target variable
X = spaceship.drop(columns=['Transported'])  # Features
y = spaceship['Transported']  # Target

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
rf_model = RandomForestClassifier(random_state=42)

# Define the parameter grid for tuning
param_grid = {
    'n_estimators': [50, 100, 200],   # Number of trees
    'max_depth': [10, 20, None],      # Depth of trees
    'min_samples_split': [2, 5, 10],  # Min samples required to split a node
    'min_samples_leaf': [1, 2, 4]     # Min samples required in a leaf node
}

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

grid_search.fit(X_train, y_train)  # Train the model

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate on test set
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Print results
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Test Accuracy after tuning: {accuracy:.4f}")

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.6s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.6s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.6s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.7s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.6s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.0s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.0s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.0s
[CV] END max_depth=10, min_samples

- Evaluate your model

In [6]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

print("Classification Report:")
print(classification_report(y_test, y_pred))

Model Accuracy: 0.7838
Confusion Matrix:
[[674 187]
 [189 689]]
Classification Report:
              precision    recall  f1-score   support

       False       0.78      0.78      0.78       861
        True       0.79      0.78      0.79       878

    accuracy                           0.78      1739
   macro avg       0.78      0.78      0.78      1739
weighted avg       0.78      0.78      0.78      1739



**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],   # Number of trees in the forest
    'max_depth': [10, 20, None],      # Depth of trees
    'min_samples_split': [2, 5, 10],  # Min samples required to split a node
    'min_samples_leaf': [1, 2, 4]     # Min samples in a leaf node
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=2, scoring='accuracy')


- Run Grid Search

In [8]:
grid_search.fit(X_train, y_train)

print(f"Best Hyperparameters: {grid_search.best_params_}")
print(f"Best Accuracy: {grid_search.best_score_:.4f}")

best_model = grid_search.best_estimator_

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.6s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.7s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.7s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.7s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.7s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.0s
[CV] END max_depth=10, min_samples

- Evaluate your model

In [11]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

y_pred = best_model.predict(X_test)  
accuracy = accuracy_score(y_test, y_pred)
print(f"🔹 Model Accuracy: {accuracy:.4f}")

conf_matrix = confusion_matrix(y_test, y_pred)
print("\n🔹 Confusion Matrix:")
print(conf_matrix)

print("\n🔹 Classification Report:")
print(classification_report(y_test, y_pred)) 

if hasattr(best_model, "predict_proba"):  
    y_prob = best_model.predict_proba(X_test)[:, 1]  
    auc_score = roc_auc_score(y_test, y_prob)
    print(f"\n🔹 ROC-AUC Score: {auc_score:.4f}")

🔹 Model Accuracy: 0.7838

🔹 Confusion Matrix:
[[674 187]
 [189 689]]

🔹 Classification Report:
              precision    recall  f1-score   support

       False       0.78      0.78      0.78       861
        True       0.79      0.78      0.79       878

    accuracy                           0.78      1739
   macro avg       0.78      0.78      0.78      1739
weighted avg       0.78      0.78      0.78      1739


🔹 ROC-AUC Score: 0.8609
